# Remote Renewable Energy Hub

## The aim of this notebook is to launch Jocelyn's Belgium model 


In [16]:
import os
import sys

project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))

belgian_data_folder = os.path.join(project_root, 'Data', 'Belgian_model')
plot_folder = os.path.join(project_root, 'Plots')
simulation_folder = os.path.join(project_root, 'Simulations')
result_folder = os.path.join(project_root, 'Results')
template_folder = os.path.join(project_root, 'Templates')
model_folder = os.path.join(project_root, 'Models GBOML')

 
sys.path.append(os.path.join(project_root, 'Modules'))

import GBOML_function as gf
import process_funct as pf

current_dir = os.getcwd()
result_dir = os.path.join(current_dir, 'Results')

# Set up

### Class and functions

In [17]:
import argparse
import sys
import json
import time
from termcolor import colored
import logging
import sys
import pandas as pd
from tqdm import tqdm
from copy import deepcopy 
from gboml import GbomlGraph
from gboml.output import write_csv
from gboml.compiler import parse_file
from gboml.compiler.classes import Parameter, Expression, Constraint, Identifier, Node

class MyGbomlGraph(GbomlGraph):
    def update_global_parameter(self, param: Parameter):
        self.global_parameters = [x if x.name != param.name else param for x in self.global_parameters]
    def get_node_by_name(self, name: str):
        for node in self.list_nodes:
            if node.name == name:
                return node
        return None
    def get_hyperedge_by_name(self, name: str):
        for h in self.list_hyperedges:
            if h.name == name:
                return h
        return None
    
    def export_model_to_lp(self, filename: str):
        """
        Optional method: exports model to LP format (requires Gurobi model to be built first).
        """
        if hasattr(self, 'model'):  # Gurobi model exists after build_model()
            self.model.write(filename)
            print(f"✅ Model written to {filename}")
        else:
            print("⚠️ Gurobi model not found. Make sure build_model() was called.")

def get_named_symbol_in_list(l, name):
    for h in l:
        if h.name == name:
            return h
    return None

def get_node_in(element, name: str):
    if isinstance(element, GbomlGraph):
        l = element.list_nodes
    elif isinstance(element, Node):
        l = element.nodes
    else:
        raise Exception(f"Unknown class {element.__class__}")
    return get_named_symbol_in_list(l, name)
    

def get_hyperedge_in(element, name: str):
    if isinstance(element, GbomlGraph):
        l = element.list_hyperedges
    elif isinstance(element, Node):
        l = element.hyperedges
    else:
        raise Exception(f"Unknown class {element.__class__}")
    return get_named_symbol_in_list(l, name)

def update_parameter(symb, param):
    symb.parameters = [x if x.name != param.name else param for x in symb.parameters] 

### Set name period


In [18]:
def set_periodicity(TimeHorizon):
    
    if TimeHorizon >= 8760:
        if TimeHorizon % 8760 == 0:
            time = f"_{int(TimeHorizon/8760)}years.json"

        elif TimeHorizon % 8760 != 0:
            time = f"_{TimeHorizon}hours.json"
                
    if TimeHorizon < 8760:
            if TimeHorizon % 24 == 0:
                time = f"_{int(TimeHorizon/24)}days.json"
            
            else:
                time = f"_{TimeHorizon}hours.json"
                
    return time

### Change parameters

In [19]:
def update_node_parameter(model, nodes_to_modify, new_capacity_value):
    """
    Updates the parameter for a list of nodes in a GBOML model.

    :param model: The loaded GBOML model (MyGbomlGraph)
    :param nodes_to_modify: List of node names (strings) to modify
    :param new_capacity_value: New value to assign to the 'max_capacity' parameter
    """
    for node_name in nodes_to_modify:
        node = model.get_node_by_name(node_name)
        if node is not None:
            param_updated = Parameter(name="max_capacity", value=new_capacity_value)
            update_parameter(node, param_updated)
            print(f"✅ max_capacity of {node_name} set to {new_capacity_value}")
        else:
            print(f"⚠️ Node {node_name} not found in the model.")
            
            
def update_subnode_parameter(model, cluster_name, subnode_names, param_name, new_value):
    """
    Updates a parameter for a list of subnodes within a given cluster node in a GBOML model.

    :param model: The loaded GBOML model (MyGbomlGraph)
    :param cluster_name: Name of the cluster (e.g., 'INLAND')
    :param subnode_names: List of subnode names (e.g., ['DME_PLANTS'])
    :param param_name: Parameter to update (e.g., 'max_capacity')
    :param new_value: Value to assign (should be a number)
    """
    cluster = model.get_node_by_name(cluster_name)
    if cluster is None:
        print(f"❌ Cluster node '{cluster_name}' not found.")
        return

    for subnode in cluster.nodes:
        if subnode.name in subnode_names:
            param = Parameter(param_name, Expression("literal", new_value))  # ✅ Fix: no str()
            update_parameter(subnode, param)
            print(f"✅ {cluster_name}.{subnode.name}: {param_name} set to {new_value}")


### Build model

In [20]:
def build_model(TimeHorizon,model_name):
    program = parse_file(model_name)
    model = MyGbomlGraph()
    model.add_nodes_in_model(*deepcopy(program.get_nodes()))
    model.add_hyperedges_in_model(*deepcopy(program.get_links()))
    model.add_global_parameters(deepcopy(program.get_global_parameters()))
    model.set_timehorizon(TimeHorizon)

    return model


def build_and_solve(model, outfile=None):
    """
    Builds and solves the given GBOML model using Gurobi.
    If the solution is optimal, it exports a JSON and CSV with results.
    If infeasible, it prints the issue and exports a .lp file for debugging.

    :param model: GBOML model (MyGbomlGraph)
    :param outfile: Optional output filename (JSON). CSV will use same name.
    :return: Dictionary of results (or empty if infeasible)
    """
    print("🚧 Building model...")
    model.build_model(8)

    # Export LP version of the model for manual debug if needed
    model.export_model_to_lp("debug_model.lp")
    print("📤 Model exported to 'debug_model.lp'")

    print("🚀 Solving...")
    solution, objective, status, solver_info, cai, vai = model.solve_gurobi(
        details=os.path.join(model_folder, "gurobi_detail.txt"),
        opt_file=os.path.join(model_folder, "gurobi.opt")
    )

    print("🧠 Gurobi status:", status)
    if "OPTIMAL" not in str(status).upper():
        print("❌ No feasible solution found.")
        return {}

    print("✅ Optimal solution found.")

    out = model.turn_solution_to_dictionary(solver_info, status, solution, objective, cai, vai)
    outlist = model.turn_solution_to_list(solution, constraints_info=cai)
    list_names = [x[0] for x in outlist]
    list_val = [x[1] for x in outlist]

    if outfile:
        with open(outfile, 'w') as f_json:
            json.dump(out, f_json, indent=4, sort_keys=True)
        write_csv(outfile.replace(".json", ".csv"), list_names, list_val, transpose=True)
        print(f"📁 Results saved: {outfile} and CSV version.")

    return out

### Scenario maker


In [21]:
def scenario_maker(clusters, nodes, params):
    """
    Generates a list of dictionaries (modifs) to apply parameter modifications
    to a combination of clusters and subnodes in a GBOML model.

    :param clusters: list of cluster names (e.g., ["INLAND", "ZEEBRUGGE"])
    :param nodes: list of subnode names (e.g., ["DME_PLANTS", "ETH_PLANTS"])
    :param params: dictionary of parameters to modify (e.g., {"max_capacity": 0})
    :return: list of formatted dictionaries for scenario modification
    """
    modifs = []
    for cluster in clusters:
        for node in nodes:
            modifs.append({
                "cluster": cluster,
                "node": node,
                "params": params.copy()
            })
    return modifs



def run_scenarios(base_model, file_path, period, new_scenario=None, modifications=None, scenario_name=None):
    """
    Runs a GBOML simulation scenario. Optionally modifies certain node parameters.

    :param base_model: Compiled GBOML model (MyGbomlGraph)
    :param file_path: Path to save output files (with trailing slash)
    :param period: Suffix for the output file (e.g. _2030)
    :param new_scenario: Optional name of the scenario (used as suffix in filenames)
    :param modifications: Optional list of parameter changes (only if new_scenario is not None). Format:
                          [{"cluster": "INLAND", "node": "DME_PLANTS", "params": {"max_capacity": 0}}]
    """
    from copy import deepcopy
    import time

    if new_scenario:
        scenario = new_scenario
    elif scenario_name:
        scenario = scenario_name
    else:
        scenario = "base" 
    
    print(f"\n🔁 Running scenario: " + colored(f'{scenario}', 'white', 'on_blue'))
    start_time = time.time()

    model = deepcopy(base_model)

    if new_scenario and modifications:
        for item in modifications:
            cluster_name = item["cluster"]
            node_name = item["node"]
            for param_name, value in item["params"].items():
                try:
                    update_subnode_parameter(model, cluster_name, [node_name], param_name, value)
                except Exception as e:
                    print(colored(f"⚠️ Skipped modification: {cluster_name}.{node_name}.{param_name} – {e}", 'yellow'))

    out_path = os.path.join(file_path, f"scenario_{scenario}_{period}.json")
    results = build_and_solve(model, outfile=out_path)

    duration = time.time() - start_time
    print(f"✅ Scenario '{scenario}' completed in {duration:.1f} seconds.")

    return results

# MAIN

- Timehorizon setting

In [22]:
TimeHorizon = 8760
# TimeHorizon = 1000 # 1 week
# TimeHorizon = 24*31 # 1 week

## DME Implementation

### DME production in RREH

In [23]:
file_dme_rreh_import = "scenario_dme_rreh_import.txt"

period = set_periodicity(TimeHorizon) 
model_dme_rreh_import = build_model(TimeHorizon, os.path.join(model_folder, file_dme_rreh_import))

In [24]:
rreh_dme_scenario_import = run_scenarios(model_dme_rreh_import, file_path=simulation_folder, period=period, scenario_name='dme_rreh_import')


🔁 Running scenario: dme_rreh_import
🚧 Building model...
Check variables of node BELGIUM : --- 0.0 seconds ---
Check variables of node DME_STORAGE : --- 0.0 seconds ---
Check variables of node SHIP_DME : --- 0.0 seconds ---
Check variables of node ALGERIA_RREH : --- 0.0 seconds ---
Check variables of node PRODUCTION : --- 0.0 seconds ---
Check variables of node BALANCE : --- 0.0 seconds ---
Check variables of node DME_STORAGE : --- 0.0 seconds ---
Check variables of node DME_PLANTS : --- 0.0 seconds ---
Check variables of node PCCC_DME : --- 0.0 seconds ---
Check variables of node CO2_STORAGE : --- 0.0 seconds ---
Check variables of node DAC : --- 0.0 seconds ---
Check variables of node H2O_STORAGE : --- 0.0 seconds ---
Check variables of node DESALINATION_PLANTS : --- 0.0 seconds ---
Check variables of node ELECTROLYSIS_PLANTS : --- 0.0 seconds ---
Check variables of node H2_STORAGE : --- 0.01517176628112793 seconds ---
Check variables of node HVDC : --- 0.0 seconds ---
Check variable

### DME production in Belgium with H2 from RREH and Belgium

In [25]:
file_dme_rreh_local = "scenario_dme_rreh_local.txt"

period = set_periodicity(TimeHorizon) 
model_dme_rreh_local = build_model(TimeHorizon, os.path.join(model_folder, file_dme_rreh_local))

In [26]:
rreh_dme_scenario_local = run_scenarios(model_dme_rreh_local, file_path=simulation_folder, period=period, scenario_name='dme_rreh_local')


🔁 Running scenario: dme_rreh_local
🚧 Building model...
Check variables of node BELGIUM : --- 0.0 seconds ---
Check variables of node BALANCE : --- 0.0 seconds ---
Check variables of node DME_STORAGE : --- 0.0 seconds ---
Check variables of node DME_PLANTS : --- 0.0010573863983154297 seconds ---
Check variables of node PCCC_DME : --- 0.0 seconds ---
Check variables of node CO2_STORAGE : --- 0.0 seconds ---
Check variables of node DAC : --- 0.0 seconds ---
Check variables of node H2O_STORAGE : --- 0.0011126995086669922 seconds ---
Check variables of node DESALINATION_PLANTS : --- 0.0 seconds ---
Check variables of node H2_GASIFICATION : --- 0.0 seconds ---
Check variables of node ELECTROLYSIS_PLANTS : --- 0.0 seconds ---
Check variables of node H2_STORAGE : --- 0.001001119613647461 seconds ---
Check variables of node BATTERIES : --- 0.0 seconds ---
Check variables of node WIND_ONSHORE : --- 0.0 seconds ---
Check variables of node PV : --- 0.0 seconds ---
Check hyperlink INLAND_DME_BALAN

### DME production in Belgium with only H2 from RREH and Belgium


In [27]:
# file_dme_rreh_local_h2_only_from_rreh = "scenario_dme_rreh_local_without_h2_prod_in_belgium.txt"

# period = set_periodicity(TimeHorizon) 
# model_dme_rreh_local_h2_only_from_rreh = build_model(TimeHorizon, os.path.join(model_folder, file_dme_rreh_local_h2_only_from_rreh))

In [28]:
# rreh_dme_scenario_local_h2_only_from_rreh = run_scenarios(model_dme_rreh_local_h2_only_from_rreh, file_path=simulation_folder, period=period, scenario_name='dme_rreh_local_h2_only_from_rreh')

### DME production in Belgium vs in RREH with H2 import from RREH

In [29]:
file_dme_rreh_import_vs_local = "scenario_dme_rreh_import_vs_local.txt"

period = set_periodicity(TimeHorizon) 
model_dme_rreh_import_vs_local = build_model(TimeHorizon, os.path.join(model_folder, file_dme_rreh_import_vs_local))

In [30]:
rreh_dme_scenario_import_vs_local = run_scenarios(model_dme_rreh_import_vs_local, file_path=simulation_folder, period=period, scenario_name='dme_rreh_import_vs_local')


🔁 Running scenario: dme_rreh_import_vs_local
🚧 Building model...
Check variables of node BELGIUM : --- 0.0 seconds ---
Check variables of node BALANCE : --- 0.0 seconds ---
Check variables of node DME_STORAGE : --- 0.0 seconds ---
Check variables of node DME_PLANTS : --- 0.0 seconds ---
Check variables of node PCCC_DME : --- 0.0 seconds ---
Check variables of node CO2_STORAGE : --- 0.0 seconds ---
Check variables of node DAC : --- 0.0 seconds ---
Check variables of node H2O_STORAGE : --- 0.0 seconds ---
Check variables of node DESALINATION_PLANTS : --- 0.0 seconds ---
Check variables of node H2_GASIFICATION : --- 0.0 seconds ---
Check variables of node ELECTROLYSIS_PLANTS : --- 0.0 seconds ---
Check variables of node H2_STORAGE : --- 0.0 seconds ---
Check variables of node BATTERIES : --- 0.0 seconds ---
Check variables of node WIND_ONSHORE : --- 0.0 seconds ---
Check variables of node PV : --- 0.0 seconds ---
Check hyperlink INLAND_DME_BALANCE : --- 0.0 seconds ---
Check hyperlink IN

# TEST

### RREH DME


In [8]:
rreh_dme_scenario = run_scenarios(model_dme_rreh, file_path=simulation_folder, period=period, scenario_name='complete_rreh_dme')


🔁 Running scenario: complete_rreh_dme
🚧 Building model...
Check variables of node BELGIUM : --- 0.0 seconds ---
Check variables of node BALANCE : --- 0.0 seconds ---
Check variables of node DME_STORAGE : --- 0.0 seconds ---
Check variables of node DME_PLANTS : --- 0.0 seconds ---
Check variables of node PCCC_DME : --- 0.0 seconds ---
Check variables of node CO2_STORAGE : --- 0.0 seconds ---
Check variables of node DAC : --- 0.0 seconds ---
Check variables of node H2O_STORAGE : --- 0.0 seconds ---
Check variables of node DESALINATION_PLANTS : --- 0.0 seconds ---
Check variables of node H2_GASIFICATION : --- 0.0 seconds ---
Check variables of node ELECTROLYSIS_PLANTS : --- 0.0 seconds ---
Check variables of node H2_STORAGE : --- 0.0 seconds ---
Check variables of node BATTERIES : --- 0.0 seconds ---
Check variables of node WIND_ONSHORE : --- 0.008149862289428711 seconds ---
Check variables of node PV : --- 0.0 seconds ---
Check hyperlink INLAND_DME_BALANCE : --- 0.0 seconds ---
Check hy

### Model 3 clusters from article

In [9]:
three_clusters_efuel_INLAND_scenario = run_scenarios(model_3_clusters_efuel_INLAND, file_path=simulation_folder, period=period, scenario_name='efuel_INLAND_ZEEBRUGGE')


🔁 Running scenario: efuel_INLAND_ZEEBRUGGE
🚧 Building model...
Check variables of node INLAND : --- 0.0 seconds ---
Check variables of node DEMAND : --- 0.0 seconds ---
Check variables of node BALANCE : --- 0.0 seconds ---
Check variables of node CO2_EXPORT : --- 0.0 seconds ---
Check variables of node CO2_STORAGE : --- 0.0 seconds ---
Check variables of node PCCC_SMR : --- 0.0 seconds ---
Check variables of node PCCC_OCGT : --- 0.0 seconds ---
Check variables of node PCCC_CCGT : --- 0.0 seconds ---
Check variables of node PCCC_CHP : --- 0.0 seconds ---
Check variables of node PCCC_WS : --- 0.0 seconds ---
Check variables of node PCCC_BM : --- 0.0 seconds ---
Check variables of node DAC : --- 0.0 seconds ---
Check variables of node PCCC_MEOH : --- 0.0 seconds ---
Check variables of node PCCC_FT : --- 0.0 seconds ---
Check variables of node PCCC_DME : --- 0.0 seconds ---
Check variables of node METHANOL_STORAGE : --- 0.0 seconds ---
Check variables of node METHANOL_PLANTS : --- 0.0 sec

In [10]:
clusters = ["INLAND", "ZEEBRUGGE"]
nodes = [ "FT_PLANTS", "REFINERY", "REFINED_PETROL_STORAGE", "PETROL_STORAGE"]
# nodes = ["ETH_PLANTS","BIOETH_PLANTS", "CELLETH_PLANTS", "ETHANOL_STORAGE"]
params = {"max_capacity": 0, "pre_installed_capacity": 0}

modifs = scenario_maker(clusters, nodes, params)
modifs_table = pf.transform_dict_into_table_several_column(modifs)

,cluster,node,params
0,INLAND,FT_PLANTS,"{'max_capacity': 0, 'pre_installed_capacity': 0}"
1,INLAND,REFINERY,"{'max_capacity': 0, 'pre_installed_capacity': 0}"
2,INLAND,REFINED_PETROL_STORAGE,"{'max_capacity': 0, 'pre_installed_capacity': 0}"
3,INLAND,PETROL_STORAGE,"{'max_capacity': 0, 'pre_installed_capacity': 0}"
4,ZEEBRUGGE,FT_PLANTS,"{'max_capacity': 0, 'pre_installed_capacity': 0}"
5,ZEEBRUGGE,REFINERY,"{'max_capacity': 0, 'pre_installed_capacity': 0}"
6,ZEEBRUGGE,REFINED_PETROL_STORAGE,"{'max_capacity': 0, 'pre_installed_capacity': 0}"
7,ZEEBRUGGE,PETROL_STORAGE,"{'max_capacity': 0, 'pre_installed_capacity': 0}"


In [11]:
three_clusters_efuel_INLAND_scenario = run_scenarios(model_3_clusters_efuel_INLAND, file_path=simulation_folder, period=period, new_scenario="efuel_INLAND_ZEEBRUGGE_no_ft", modifications=modifs)


🔁 Running scenario: efuel_INLAND_ZEEBRUGGE_no_ft
✅ INLAND.FT_PLANTS: max_capacity set to 0
✅ INLAND.FT_PLANTS: pre_installed_capacity set to 0
✅ INLAND.REFINERY: max_capacity set to 0
✅ INLAND.REFINERY: pre_installed_capacity set to 0
✅ INLAND.REFINED_PETROL_STORAGE: max_capacity set to 0
✅ INLAND.REFINED_PETROL_STORAGE: pre_installed_capacity set to 0
✅ INLAND.PETROL_STORAGE: max_capacity set to 0
✅ INLAND.PETROL_STORAGE: pre_installed_capacity set to 0
✅ ZEEBRUGGE.FT_PLANTS: max_capacity set to 0
✅ ZEEBRUGGE.FT_PLANTS: pre_installed_capacity set to 0
✅ ZEEBRUGGE.REFINERY: max_capacity set to 0
✅ ZEEBRUGGE.REFINERY: pre_installed_capacity set to 0
✅ ZEEBRUGGE.REFINED_PETROL_STORAGE: max_capacity set to 0
✅ ZEEBRUGGE.REFINED_PETROL_STORAGE: pre_installed_capacity set to 0
✅ ZEEBRUGGE.PETROL_STORAGE: max_capacity set to 0
✅ ZEEBRUGGE.PETROL_STORAGE: pre_installed_capacity set to 0
🚧 Building model...
Check variables of node INLAND : --- 0.0 seconds ---
Check variables of node DEMAND : 

### Model 4 clusters

In [9]:
four_clusters_scenario = run_scenarios(model_4_clusters, file_path=simulation_folder, period=period, scenario_name='4_clusters_efuel_only_rreh')


🔁 Running scenario: 4_clusters_efuel_only_rreh
🚧 Building model...
Check variables of node INLAND : --- 0.0 seconds ---
Check variables of node BALANCE : --- 0.0 seconds ---
Check variables of node CO2_EXPORT : --- 0.0 seconds ---
Check variables of node CO2_STORAGE : --- 0.0010004043579101562 seconds ---
Check variables of node PCCC_SMR : --- 0.0 seconds ---
Check variables of node PCCC_OCGT : --- 0.0 seconds ---
Check variables of node PCCC_CCGT : --- 0.0 seconds ---
Check variables of node PCCC_CHP : --- 0.0 seconds ---
Check variables of node PCCC_WS : --- 0.0 seconds ---
Check variables of node PCCC_BM : --- 0.0 seconds ---
Check variables of node DAC : --- 0.0028939247131347656 seconds ---
Check variables of node H2_STORAGE : --- 0.0005266666412353516 seconds ---
Check variables of node SMR : --- 0.0 seconds ---
Check variables of node H2_INTERCONNECTION_NL : --- 0.0 seconds ---
Check variables of node H2_INTERCONNECTION_DE : --- 0.0 seconds ---
Check variables of node H2_INTERC

### Model from the article 

In [8]:
base_scenario = run_scenarios(model_efuel_competing, file_path=simulation_folder, period=period, scenario_name='base_from_article')


🔁 Running scenario: base_from_article
🚧 Building model...
Check variables of node INLAND : --- 0.0 seconds ---
Check variables of node BALANCE : --- 0.0 seconds ---
Check variables of node H2O_STORAGE : --- 0.0 seconds ---
Check variables of node CO2_EXPORT : --- 0.0 seconds ---
Check variables of node CO2_STORAGE : --- 0.0 seconds ---
Check variables of node PCCC_SMR : --- 0.0 seconds ---
Check variables of node PCCC_OCGT : --- 0.0 seconds ---
Check variables of node PCCC_CCGT : --- 0.0 seconds ---
Check variables of node PCCC_CHP : --- 0.0 seconds ---
Check variables of node PCCC_WS : --- 0.0 seconds ---
Check variables of node PCCC_BM : --- 0.0 seconds ---
Check variables of node PCCC_DME : --- 0.0 seconds ---
Check variables of node DAC : --- 0.0 seconds ---
Check variables of node H2_STORAGE : --- 0.0 seconds ---
Check variables of node SMR : --- 0.0 seconds ---
Check variables of node H2_INTERCONNECTION_NL : --- 0.0 seconds ---
Check variables of node H2_INTERCONNECTION_DE : ---

### No e_fuel integration

In [9]:
clusters = ["INLAND", "ZEEBRUGGE"]
nodes = ["DME_PLANTS", "ETH_PLANTS", "FT_PLANTS", "METHANOL_PLANTS", "BIOETH_PLANTS", "CELLETH_PLANTS",
               "DME_storage", "ethanol_storage", "REFINERY", "REFINED_PETROL_STORAGE", "PETROL_STORAGE", "METHANOL_STORAGE"]
params = {"max_capacity": 0, "pre_installed_capacity": 0}

modifs = scenario_maker(clusters, nodes, params)
modifs_table = pf.transform_dict_into_table_several_column(modifs)

,cluster,node,params
0,INLAND,DME_PLANTS,"{'max_capacity': 0, 'pre_installed_capacity': 0}"
1,INLAND,ETH_PLANTS,"{'max_capacity': 0, 'pre_installed_capacity': 0}"
2,INLAND,FT_PLANTS,"{'max_capacity': 0, 'pre_installed_capacity': 0}"
3,INLAND,METHANOL_PLANTS,"{'max_capacity': 0, 'pre_installed_capacity': 0}"
4,INLAND,BIOETH_PLANTS,"{'max_capacity': 0, 'pre_installed_capacity': 0}"
5,INLAND,CELLETH_PLANTS,"{'max_capacity': 0, 'pre_installed_capacity': 0}"
6,INLAND,DME_storage,"{'max_capacity': 0, 'pre_installed_capacity': 0}"
7,INLAND,ethanol_storage,"{'max_capacity': 0, 'pre_installed_capacity': 0}"
8,INLAND,REFINERY,"{'max_capacity': 0, 'pre_installed_capacity': 0}"
9,INLAND,REFINED_PETROL_STORAGE,"{'max_capacity': 0, 'pre_installed_capacity': 0}"


In [9]:
no_efuels_scenario = run_scenarios(model_efuel_competing, file_path=simulation_folder, period=period, new_scenario="no_efuel", modifications=modifs)


🔁 Running scenario: no_efuel
✅ INLAND.DME_PLANTS: max_capacity set to 0
✅ INLAND.DME_PLANTS: pre_installed_capacity set to 0
✅ INLAND.ETH_PLANTS: max_capacity set to 0
✅ INLAND.ETH_PLANTS: pre_installed_capacity set to 0
✅ INLAND.FT_PLANTS: max_capacity set to 0
✅ INLAND.FT_PLANTS: pre_installed_capacity set to 0
✅ INLAND.METHANOL_PLANTS: max_capacity set to 0
✅ INLAND.METHANOL_PLANTS: pre_installed_capacity set to 0
✅ INLAND.BIOETH_PLANTS: max_capacity set to 0
✅ INLAND.BIOETH_PLANTS: pre_installed_capacity set to 0
✅ INLAND.CELLETH_PLANTS: max_capacity set to 0
✅ INLAND.CELLETH_PLANTS: pre_installed_capacity set to 0
✅ INLAND.DME_storage: max_capacity set to 0
✅ INLAND.DME_storage: pre_installed_capacity set to 0
✅ INLAND.ethanol_storage: max_capacity set to 0
✅ INLAND.ethanol_storage: pre_installed_capacity set to 0
✅ INLAND.REFINERY: max_capacity set to 0
✅ INLAND.REFINERY: pre_installed_capacity set to 0
✅ INLAND.REFINED_PETROL_STORAGE: max_capacity set to 0
✅ INLAND.REFINED_PETR

### All Efuels integrated:

- DME
- Methanol
- Ethanol and bioEthanol
- FT product: Diesel, gasoline and kerosene

#### Efuels competing Natural gas and hydrogen

In [10]:
all_efuels_scenario = run_scenarios(model_efuel_competing, file_path=simulation_folder, period=period)


🔁 Running scenario: base
🚧 Building model...
Check variables of node INLAND : --- 0.004001140594482422 seconds ---
Check variables of node BALANCE : --- 0.0 seconds ---
Check variables of node H2O_STORAGE : --- 0.0 seconds ---
Check variables of node CO2_EXPORT : --- 0.0 seconds ---
Check variables of node CO2_STORAGE : --- 0.0 seconds ---
Check variables of node PCCC_SMR : --- 0.0009989738464355469 seconds ---
Check variables of node PCCC_OCGT : --- 0.0 seconds ---
Check variables of node PCCC_CCGT : --- 0.0 seconds ---
Check variables of node PCCC_CHP : --- 0.0 seconds ---
Check variables of node PCCC_WS : --- 0.0 seconds ---
Check variables of node PCCC_BM : --- 0.0 seconds ---
Check variables of node PCCC_DME : --- 0.00099945068359375 seconds ---
Check variables of node DAC : --- 0.0 seconds ---
Check variables of node H2_STORAGE : --- 0.0 seconds ---
Check variables of node SMR : --- 0.0 seconds ---
Check variables of node H2_INTERCONNECTION_NL : --- 0.0 seconds ---
Check variabl

#### Efuels replacing some natural gas and hydrogen demands

In [13]:
all_efuels_scenario = run_scenarios(model_efuel_replacing, file_path=simulation_folder, period=period)


🔁 Running scenario: base
🚧 Building model...
Check variables of node INLAND : --- 0.0 seconds ---
Check variables of node BALANCE : --- 0.0 seconds ---
Check variables of node H2O_STORAGE : --- 0.0 seconds ---
Check variables of node CO2_EXPORT : --- 0.0 seconds ---
Check variables of node CO2_STORAGE : --- 0.0 seconds ---
Check variables of node PCCC_SMR : --- 0.0 seconds ---
Check variables of node PCCC_OCGT : --- 0.0 seconds ---
Check variables of node PCCC_CCGT : --- 0.0 seconds ---
Check variables of node PCCC_CHP : --- 0.0 seconds ---
Check variables of node PCCC_WS : --- 0.0 seconds ---
Check variables of node PCCC_BM : --- 0.0 seconds ---
Check variables of node PCCC_DME : --- 0.0 seconds ---
Check variables of node DAC : --- 0.0 seconds ---
Check variables of node H2_STORAGE : --- 0.0 seconds ---
Check variables of node SMR : --- 0.008013486862182617 seconds ---
Check variables of node H2_INTERCONNECTION_NL : --- 0.0 seconds ---
Check variables of node H2_INTERCONNECTION_DE :

## TEST

In [ ]:
clusters = ["INLAND", "ZEEBRUGGE"]
efuel_nodes = ["DME_PLANTS", "ETH_PLANTS", "FT_PLANTS", "METHANOL_PLANTS", "BIOETH_PLANTS", "CELLETH_PLANTS",
               "DME_storage", "ethanol_storage", "REFINERY", "REFINED_PETROL_STORAGE", "PETROL_STORAGE", "METHANOL_STORAGE"]
params_to_set = {"max_capacity": 0}

run_scenarios(base_model, file_path, period,
              clusters=clusters,
              subnodes=efuel_nodes,
              parameters=params_to_set)

In [ ]:
run_scenarios(base_model = base_model, file_path = simulation_folder, period = period) 

🚀 Starting scenario simulations...



Scenarios:   0%|          | 0/2 [00:00<?, ?it/s]


🔁 Running scenario: No efuels
✅ max_capacity in subnode INLAND.DME_PLANTS set to 0
✅ max_capacity in subnode INLAND.ETH_PLANTS set to 0
✅ max_capacity in subnode INLAND.FT_PLANTS set to 0
✅ max_capacity in subnode INLAND.METHANOL_PLANTS set to 0
✅ max_capacity in subnode INLAND.BIOETH_PLANTS set to 0
✅ max_capacity in subnode INLAND.CELLETH_PLANTS set to 0
🚧 Building model...
Check variables of node INLAND : --- 0.0 seconds ---
Check variables of node BALANCE : --- 0.0 seconds ---
Check variables of node H2O_STORAGE : --- 0.0 seconds ---
Check variables of node CO2_EXPORT : --- 0.0 seconds ---
Check variables of node CO2_STORAGE : --- 0.0 seconds ---
Check variables of node PCCC_SMR : --- 0.0 seconds ---
Check variables of node PCCC_OCGT : --- 0.00801706314086914 seconds ---
Check variables of node PCCC_CCGT : --- 0.0 seconds ---
Check variables of node PCCC_CHP : --- 0.0 seconds ---
Check variables of node PCCC_WS : --- 0.0 seconds ---
Check variables of node PCCC_BM : --- 0.0 second

Scenarios:  50%|█████     | 1/2 [42:16<42:16, 2536.41s/it]

📁 Results saved: C:/Users/yaita/OneDrive/Bureau/Master Thesis/Belgium Model - Younès/Simulations/scenario_no_efuels_1years.json.json and CSV version.
✅ Scenario 'no_efuels' completed in 2536.4 seconds.

🔁 Running scenario: With all efuels
🚧 Building model...
Check variables of node INLAND : --- 0.0 seconds ---
Check variables of node BALANCE : --- 0.0 seconds ---
Check variables of node H2O_STORAGE : --- 0.0 seconds ---
Check variables of node CO2_EXPORT : --- 0.0 seconds ---
Check variables of node CO2_STORAGE : --- 0.015127182006835938 seconds ---
Check variables of node PCCC_SMR : --- 0.0 seconds ---
Check variables of node PCCC_OCGT : --- 0.0 seconds ---
Check variables of node PCCC_CCGT : --- 0.0 seconds ---
Check variables of node PCCC_CHP : --- 0.0 seconds ---
Check variables of node PCCC_WS : --- 0.00128936767578125 seconds ---
Check variables of node PCCC_BM : --- 0.0 seconds ---
Check variables of node PCCC_DME : --- 0.002335071563720703 seconds ---
Check variables of node DA

Scenarios: 100%|██████████| 2/2 [57:29<00:00, 1724.58s/it]

📁 Results saved: C:/Users/yaita/OneDrive/Bureau/Master Thesis/Belgium Model - Younès/Simulations/scenario_with_all_efuels_1years.json.json and CSV version.
✅ Scenario 'with_all_efuels' completed in 912.8 seconds.


In [ ]:
import pandas as pd

# Charger le fichier résumé
df = pd.read_csv("C:/Users/yaita/OneDrive/Bureau/Master Thesis/Belgium Model - Younès/Simulations/summary_results.csv")

# Renommer les colonnes pour affichage plus propre (optionnel)
df.columns = [
    "Scenario",
    "H₂ Import Cost [€/kWh]",
    "Total Cost [M€]",
    "DME Production [GWh]",
    "Methanol Production [GWh]",
]

# Convertir en tableau LaTeX
latex_table = df.to_latex(index=False, float_format="%.2f", column_format="lccccc", caption="Summary of simulation scenarios", label="tab:scenario_summary")

# Afficher dans le terminal
print(latex_table)

# Optionnel : sauvegarder dans un .tex
with open("C:/Users/yaita/OneDrive/Bureau/Master Thesis/Belgium Model - Younès/Simulations/scenario_summary_table.tex", "w") as f:
    f.write(latex_table)

print("✅ LaTeX table saved.")


# Base case without efuels

In [17]:
# name_file = "Belgium_GBOML.txt"
# TimeHorizon = 24*7    

# model = build_model(TimeHorizon,name_file)

# names_case = '_base_case_without_efuels'

# file_path = "C:/Users/yaita/OneDrive/Bureau/Master Thesis/Belgium Model - Younès/Simulations/"

# Inland = get_node_in(model, "INLAND")
# H2_NL = get_node_in(Inland, "H2_INTERCONNECTION_NL")
# h2_import_costs = 0.3 
# time = set_periodicity(TimeHorizon)
    
# h2_import_costs = [0.075,0.1,0.105,0.108,0.1125,0.125,0.1375,0.15,0.1625,0.175,0.1875,0.2,0.225,0.25,0.275,0.3]
# for cost in h2_import_costs:
#     update_parameter(H2_NL, Parameter("import_cost", Expression("literal", cost)))
#     name_outfile = file_path + "3_clusters_Belgium_" + names_case + "_cost_h2_" + str(cost) + ".json"
#     out = build_and_solve(model, outfile = name_outfile)

# update_parameter(H2_NL, Parameter("import_cost", Expression("literal", h2_import_costs)))
# name_outfile = file_path + "2050_Belgium_model_" + names_case + time 
# out = build_and_solve(model, outfile = name_outfile)


In [16]:
# update_subnode_parameter(
#     model,
#     cluster_name="INLAND",
#     subnode_names=["DME_PLANTS", "ETH_PLANTS", "FT_PLANTS", "METHANOL_PLANTS", "BIOETH_PLANTS", "CELLETH_PLANTS"],
#     param_name="max_capacity",
#     new_value=0
# )

# Base case with efuels

In [ ]:

# Charger le modèle
model = build_model(TimeHorizon=8760, model_name="Belgium_GBOML_only_h2.txt")

# Définir la liste des nœuds à modifier
nodes_to_modify = [
    "DME_PLANTS",
    "METHANOL_PLANTS",
    "ETH_PLANTS",
    "CELLETH_PLANTS",
    "BIOETH_PLANTS",
    "FT_PLANTS",
]

# Définir la nouvelle valeur (exemple : 0 pour désactivation)
new_capacity_value = 0

# Appeler la fonction
update_max_capacity(model, nodes_to_modify, new_capacity_value)

# Résoudre le modèle ensuite si besoin :
# build_and_solve(model)